In [ ]:
import tensorflow as tf
import csv
import math
from tensorflow import keras



# Helper libraries
import numpy as np

import rospy
import os
import json
import numpy as np
import random
import time
import sys
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dense, Dropout, Activation
from collections import deque

In [ ]:
batch_size = 2
train_start = 64
memory = deque(maxlen=1000000)


In [1]:

from tensorflow import keras
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model

# Headline input: meant to receive sequences of 100 integers, between 1 and 10000.
# Note that we can name any layer by passing it a "name" argument.
state = keras.layers.Input(shape=(14,), name='main_input')

# This embedding layer will encode the input sequence
# into a sequence of dense 512-dimensional vectors.
d1 = Dense(512, activation='relu')(state)
#auxiliary_output = keras.layers.Dense(1, activation='sigmoid', name='aux_output')(d1)

action = keras.layers.Input(shape=(2,), name='aux_input')
x = keras.layers.concatenate([d1, action])

x = Dense(512, activation='relu')(x)
x = Dense(512, activation='relu')(x)


# And finally we add the main logistic regression layer
q_val= Dense(1, activation='sigmoid', name='main_output')(x)

critic = Model(inputs=[state, action], outputs=[q_val])
critic.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 14)           0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 512)          7680        main_input[0][0]                 
__________________________________________________________________________________________________
aux_input (InputLayer)          (None, 2)            0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 514)          0           dense[0][0]                      
                                                                 aux_input[0][0]                  
__________

In [2]:


state = keras.layers.Input(shape=(14,), name='main_input')

# This embedding layer will encode the input sequence
# into a sequence of dense 512-dimensional vectors.
x = Dense(512, activation='relu')(state)
x = Dense(512, activation='relu')(state)
x = Dense(512, activation='relu')(state)
#auxiliary_output = keras.layers.Dense(1, activation='sigmoid', name='aux_output')(d1)

lin_vel = Dense(1, activation='sigmoid', name='lin_vel')(x)
ang_vel = Dense(1, activation='tanh', name='ang_vel')(x)
output = keras.layers.concatenate([lin_vel, ang_vel])

actor = Model(inputs=[state], outputs=[output])
actor.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 14)           0                                            
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 512)          7680        main_input[0][0]                 
__________________________________________________________________________________________________
lin_vel (Dense)                 (None, 1)            513         dense_5[0][0]                    
__________________________________________________________________________________________________
ang_vel (Dense)                 (None, 1)            513         dense_5[0][0]                    
__________________________________________________________________________________________________
concatenat

In [3]:
test = [0,0]

In [ ]:
critic.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='mse',
              metrics=['accuracy'])


In [4]:
#model.fit(ml_in, ml_out, epochs=10)
test[0] = 1

In [5]:
#predictions = model.predict(ml_in)
test

[1, 0]

In [ ]:
#predictions[0:20]
#predictions.shape



In [ ]:
model.summary()
#model.save_weights('./src/checkpoints/my_checkpoint')